In [ ]:
import json
import pandas as pd
import geopandas as gpd

import plotly.express as px

from covis.utils import get_project_root

# regional choropleth with date slider 

#### read in data for regional covid-19 deaths

In [ ]:
weekly_reg_cov = pd.read_csv(
    get_project_root() / "output/covid19_weekly_regional_deaths_2020-2022.csv",
    index_col=0
)

In [ ]:
print(weekly_reg_cov.shape)
weekly_reg_cov.head()

#### read in data for regional total deaths

In [ ]:
weekly_reg_tot = pd.read_csv(
    get_project_root() / "output/total_weekly_regional_deaths_2020-2022.csv",
    index_col=0
)

In [ ]:
print(weekly_reg_tot.shape)
weekly_reg_tot.head()

#### regional geojsons to gdf

In [ ]:
# # map regions:
# jpath = get_project_root() / "output/eer_dict.json"
# with open(jpath) as f:
#     eer = json.load(f)

# eer

In [ ]:
geo = gpd.read_parquet(
    get_project_root() / "output/EWgeo.pkl",
)

In [ ]:
geo

In [ ]:
geo = geo[["EER13NM", "geometry"]]
geo.set_index("EER13NM", inplace=True)

In [ ]:
geo

In [ ]:
geo = geo.set_crs("EPSG:27700")

In [ ]:
geo.crs

In [ ]:
weekly_reg_cov.reset_index(inplace=True)
weekly_reg_cov.rename(columns={"index":"week"}, inplace=True)

In [ ]:
weekly_reg_cov.head()

In [ ]:
weekly_reg_tot.reset_index(inplace=True)
weekly_reg_tot.rename(columns={"index":"week"}, inplace=True)

In [ ]:
weekly_reg_tot.head()

#### check same index and date markers before merge

In [ ]:
weekly_reg_cov.index.equals(weekly_reg_tot.index)

In [ ]:
weekly_reg_cov["Week ended"].equals(weekly_reg_tot["Week ended"])

In [ ]:
# weekly_reg_cov.merge(
#     weekly_reg_tot[[col for col in weekly_reg_tot.columns if col not in ["week"]]],
#     left_on="Week ended",
#     right_on="Week ended",
#     suffixes=[" covid19"," total"]
# )

#### melt to long form

In [ ]:
weekly_reg_long_cov = weekly_reg_cov.melt(
    id_vars="Week ended",
    value_vars=[col for col in weekly_reg_cov.columns if col not in ["Week ended", "week"]],
    var_name="region",
    value_name="covid19 deaths"
)

In [ ]:
weekly_reg_long_cov.head()

In [ ]:
weekly_reg_long_tot = weekly_reg_tot.melt(
    id_vars="Week ended",
    value_vars=[col for col in weekly_reg_cov.columns if col not in ["Week ended", "week"]],
    var_name="region",
    value_name="total deaths"
)

In [ ]:
weekly_reg_long_tot.head()

In [ ]:
weekly_reg_long = pd.merge(
    weekly_reg_long_cov,
    weekly_reg_long_tot,
    how="inner",
    left_on=["Week ended", "region"],
    right_on=["Week ended", "region"]
)

In [ ]:
len(weekly_reg_long) == len(weekly_reg_long_cov) == len(weekly_reg_long_tot)

In [ ]:
weekly_reg_long["pc deaths due to covid19"] = weekly_reg_long["covid19 deaths"] / weekly_reg_long["total deaths"]

In [ ]:
print(weekly_reg_long.shape)
weekly_reg_long.head()

In [ ]:
# weekly_reg_long.to_csv(
#     get_project_root() / "output/long_form_regional_weekly_deaths.csv"
# )

In [ ]:
weekly_reg_long_test = weekly_reg_long.sort_values("Week ended").reset_index().loc[100:300]

In [ ]:
fig = px.choropleth(
    weekly_reg_long_test,
    geojson=geo["geometry"],
    locations=weekly_reg_long_test["region"],
    color=weekly_reg_long_test["pc deaths due to covid19"],
    animation_frame="Week ended",
    color_continuous_scale="reds",
    range_color=(weekly_reg_long_test["pc deaths due to covid19"].min(), weekly_reg_long_test["pc deaths due to covid19"].max()),
    hover_name="region",
    hover_data=["pc deaths due to covid19", "covid19 deaths", "total deaths"]
)

fig.update_geos(
    fitbounds="geojson",
    visible=False
)

In [ ]:
# fig.write_html(
#     get_project_root() / "output/choro_slider_test.html"
# )

# fig.write_html(
#     get_project_root() / "lfs/choro_slider_test.html"
# )

* these functions will be moved into a script to produce the choropleth for the full date range